In [1]:
from sodapy import Socrata
import pandas as pd

In [2]:
client = Socrata("data.cityofnewyork.us", None)

In [3]:
dataset_id = "nre2-6m2s"

In [4]:
results = client.get(dataset_id, limit = 5000)

In [6]:
df = pd.DataFrame.from_records(results)

In [7]:
df.head()

,business_name,industry,complaint_type,mediation_start_date,mediation_close_date,complaint_result,satisfaction,restitution,business_building,business_street,building_address_unit,business_city,business_state,business_zip,complainant_zip,longitude,latitude
0,IMPARK WATER LLC,Garage - 049,Advertising/General - A01,2018-06-13T00:00:00.000,2018-07-03T00:00:00.000,Referred to Outside - RTO,NA,0.00,307,7TH AVE,STE 301,NEW YORK,NY,10001,08721,-74.00978122562515,40.7035970814381
1,SOGO NY TRAVEL & TOUR INC.,Travel Agency - 440,Exchange Goods/Contract Cancelled - E01,2014-10-17T00:00:00.000,2014-11-18T00:00:00.000,Cash Amount - AMT,Yes,438.00,3719,MAIN ST,2C,FLUSHING,NY,11354,10026,-73.8309544651437,40.76123957174321
2,"CONTINENTAL SERVICE GROUP, INC.",Debt Collection Agency - 122,Billing Dispute - B02,2015-04-29T00:00:00.000,2015-05-18T00:00:00.000,Bill Reduced - BLR,Yes,13948.00,PO BOX,7,NaN,FAIRPORT,NY,14450,10472,NaN,NaN
3,RELIABLE REMODELERS INC.,Home Improvement Contractor - 100,Damaged Goods - D01,2017-03-29T00:00:00.000,2017-05-04T00:00:00.000,Referred to Hearing - RTH,No,0.00,2516,FRANCIS LEWIS BLVD,NaN,FLUSHING,NY,11358,11422,NaN,NaN
4,170 ELECTRONICS WORLD USA INC,Electronic Store - 001,Damaged Goods - D01,2021-03-05T00:00:00.000,2021-04-09T00:00:00.000,Advised to Sue - ATS,No,0.00,56A,E 170TH ST,NaN,BRONX,NY,10452,NaN,-73.91647618548,40.83975293547475


In [24]:
df.shape

(5000, 17)

In [25]:
df.columns.tolist()

['business_name',
 'industry',
 'complaint_type',
 'mediation_start_date',
 'mediation_close_date',
 'complaint_result',
 'satisfaction',
 'restitution',
 'business_building',
 'business_street',
 'building_address_unit',
 'business_city',
 'business_state',
 'business_zip',
 'complainant_zip',
 'longitude',
 'latitude']

In [26]:
df.dtypes

business_name            object
industry                 object
complaint_type           object
mediation_start_date     object
mediation_close_date     object
complaint_result         object
satisfaction             object
restitution              object
business_building        object
business_street          object
building_address_unit    object
business_city            object
business_state           object
business_zip             object
complainant_zip          object
longitude                object
latitude                 object
dtype: object

In [14]:
nulls = df.isnull().sum()
nulls[nulls > 0].sort_values(ascending = False)

building_address_unit    4108
latitude                 2155
longitude                2155
business_street           442
complainant_zip           414
complaint_type            368
business_building         366
business_city             354
business_state            354
business_zip              354
business_name             163
satisfaction                1
dtype: int64

In [15]:
df_clean = df.dropna(subset=['complaint_type', 'business_name'])

In [16]:
# Optional: Drop all rows with ANY missing values (for a fully clean base)
df_clean = df_clean.dropna()


In [17]:
# Fill missing location fields with placeholder
df_clean['latitude'] = df_clean['latitude'].fillna(0)
df_clean['longitude'] = df_clean['longitude'].fillna(0)


In [18]:
# Drop rows missing 'complaint_type' or 'business_name'
df_clean = df.dropna(subset=['complaint_type', 'business_name'])

# Check new shape
print("Shape after dropping key nulls:", df_clean.shape)


Shape after dropping key nulls: (4479, 17)


In [21]:
# Convert to datetime using actual column names
df_clean['mediation_start_date'] = pd.to_datetime(df_clean['mediation_start_date'], errors='coerce')
df_clean['mediation_close_date'] = pd.to_datetime(df_clean['mediation_close_date'], errors='coerce')

# Create resolution time column
df_clean['resolution_days'] = (df_clean['mediation_close_date'] - df_clean['mediation_start_date']).dt.days

# Show a few examples
df_clean[['mediation_start_date', 'mediation_close_date', 'resolution_days']].head()


C:\Users\Aryaman Kachroo\AppData\Local\Temp\ipykernel_22460\2900134819.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['mediation_start_date'] = pd.to_datetime(df_clean['mediation_start_date'], errors='coerce')
C:\Users\Aryaman Kachroo\AppData\Local\Temp\ipykernel_22460\2900134819.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['mediation_close_date'] = pd.to_datetime(df_clean['mediation_close_date'], errors='coerce')
C:\Users\Aryaman Kachroo\AppData\Local\Temp\ipykernel_22460\2900

,mediation_start_date,mediation_close_date,resolution_days
0,2018-06-13,2018-07-03,20
1,2014-10-17,2014-11-18,32
2,2015-04-29,2015-05-18,19
3,2017-03-29,2017-05-04,36
4,2021-03-05,2021-04-09,35


In [20]:
print(df_clean.columns.tolist())


['business_name', 'industry', 'complaint_type', 'mediation_start_date', 'mediation_close_date', 'complaint_result', 'satisfaction', 'restitution', 'business_building', 'business_street', 'building_address_unit', 'business_city', 'business_state', 'business_zip', 'complainant_zip', 'longitude', 'latitude']


In [29]:
df.isnull().sum()

business_name             163
industry                    0
complaint_type            368
mediation_start_date        0
mediation_close_date        0
complaint_result            0
satisfaction                1
restitution                 0
business_building         366
business_street           442
building_address_unit    4108
business_city             354
business_state            354
business_zip              354
complainant_zip           414
longitude                2155
latitude                 2155
dtype: int64

In [30]:
df_clean = df.dropna(subset=['business_name', 'complaint_type'])


In [31]:
df_clean = df_clean.drop(columns=[
    'building_address_unit', 'business_building', 'business_street',
    'business_city', 'business_state', 'business_zip', 'complainant_zip'
])


In [32]:
df_clean[['latitude', 'longitude']] = df_clean[['latitude', 'longitude']].fillna(0)


In [33]:
df_clean['satisfaction'] = df_clean['satisfaction'].fillna('Unknown')


In [34]:
df_clean.isnull().sum()


business_name           0
industry                0
complaint_type          0
mediation_start_date    0
mediation_close_date    0
complaint_result        0
satisfaction            0
restitution             0
longitude               0
latitude                0
dtype: int64

In [35]:
# How vendors responded
df_clean['complaint_result'].value_counts()

complaint_result
Advised to Sue - ATS                                  777
Resolved and Consumer Satisfied - SPF                 772
Cash Amount - AMT                                     731
Bill Reduced - BLR                                    229
No Satisfactory Agreement - NSA                       203
Referred to Outside - RTO                             200
No Business Response - NVR                            183
No Consumer Response - NCR                            183
Credit Card Refund and/or Contract Cancelled - CRC    164
Referred to Hearing - RTH                             162
Consumer Took Action - CTA                            141
Goods Exchanged - GEX                                 128
Goods Repaired - GRS                                  116
Unable to Locate Business - ULV                       112
Store Credit - SCR                                    100
Complaint Invalid - CIN                                98
Goods Received - GDR                                   

In [36]:
# Satisfaction levels
df_clean['satisfaction'].value_counts()


satisfaction
Yes        2320
No         1466
NA          692
Unknown       1
Name: count, dtype: int64

In [37]:
# Define positive response keywords
positive_responses = [
    'Resolved', 'Reduced', 'Credit Card Refund', 'Goods', 'Store Credit',
    'Cash Amount', 'Took Action', 'Consumer Restitution'
]

# Create binary target variable
df_clean['vendor_responded'] = df_clean['complaint_result'].apply(
    lambda x: 1 if any(keyword.lower() in x.lower() for keyword in positive_responses) else 0
)

# Check balance
df_clean['vendor_responded'].value_counts()


vendor_responded
1    2456
0    2023
Name: count, dtype: int64

In [40]:
from sklearn.preprocessing import LabelEncoder

In [41]:
# Encode 'industry', 'complaint_type', 'satisfaction'
le_industry = LabelEncoder()
le_complaint = LabelEncoder()
le_satisfaction = LabelEncoder()

df_clean['industry_encoded'] = le_industry.fit_transform(df_clean['industry'])
df_clean['complaint_encoded'] = le_complaint.fit_transform(df_clean['complaint_type'])
df_clean['satisfaction_encoded'] = le_satisfaction.fit_transform(df_clean['satisfaction'].astype(str))

In [44]:
# Recalculate resolution_days using cleaned columns
df_clean['mediation_start_date'] = pd.to_datetime(df_clean['mediation_start_date'], errors='coerce')
df_clean['mediation_close_date'] = pd.to_datetime(df_clean['mediation_close_date'], errors='coerce')

df_clean['resolution_days'] = (df_clean['mediation_close_date'] - df_clean['mediation_start_date']).dt.days


In [45]:
print(df_clean.columns.tolist())


['business_name', 'industry', 'complaint_type', 'mediation_start_date', 'mediation_close_date', 'complaint_result', 'satisfaction', 'restitution', 'longitude', 'latitude', 'vendor_responded', 'industry_encoded', 'complaint_encoded', 'satisfaction_encoded', 'resolution_days']


In [46]:
features = ['industry_encoded', 'complaint_encoded', 'resolution_days', 'satisfaction_encoded']
X = df_clean[features]
y = df_clean['vendor_responded']


In [47]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("📊 Classification Report:")
print(classification_report(y_test, y_pred))

print("\n🧩 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


📊 Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       604
           1       0.97      0.94      0.96       740

    accuracy                           0.95      1344
   macro avg       0.95      0.96      0.95      1344
weighted avg       0.96      0.95      0.95      1344


🧩 Confusion Matrix:
[[585  19]
 [ 42 698]]
